<a href="https://colab.research.google.com/github/VinaySurwase/UkiyoFusion/blob/main/LoRA_fine_tuned_Stable_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ✅ 1. Install required libraries
!pip install --upgrade diffusers==0.26.3 transformers==4.36.2 huggingface_hub==0.20.3 datasets accelerate==0.27.2 -q
!pip install peft==0.7.1 --no-deps


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.1/330.1 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [ ]:
# ✅ 2. Mount Google Drive to save model
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
# ✅ 3. Import libs
import torch
from diffusers import StableDiffusionPipeline, UNet2DConditionModel
from peft import get_peft_model, LoraConfig, TaskType
from transformers import CLIPTokenizer
from huggingface_hub import notebook_login
from datasets import load_dataset

/usr/local/lib/python3.11/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:
# ✅ 4. Set model & LoRA config
model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    safety_checker=None,
    variant="fp16",
).to("cuda")

pipe.enable_attention_slicing()  # helps with low VRAM

unet = pipe.unet
unet.train()

lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["to_q", "to_k", "to_v", "to_out.0"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.FEATURE_EXTRACTION,  # Using FEATURE_EXTRACTION as a generic task type
)

unet = get_peft_model(unet, lora_config)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [ ]:
# ✅ 5. Load your dataset images from Drive
# Create folder in Drive: /content/drive/MyDrive/ukiyoe_lora/images
import os
from PIL import Image

image_folder = "/content/drive/MyDrive/ukiyoe_lora/images"
image_paths = [os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.lower().endswith(("jpg", "png"))]

# Helper: preprocess
def preprocess_image(image_path, image_size=512):
    image = Image.open(image_path).convert("RGB").resize((image_size, image_size))
    return pipe.feature_extractor(images=image, return_tensors="pt").pixel_values[0]

In [ ]:
# ✅ 6. Prepare dataset
images = torch.stack([preprocess_image(p) for p in image_paths]).to("cuda", dtype=torch.float16)

# Text prompt conditioning
prompt = "a beautiful ukiyo-e artwork"
text_inputs = pipe.tokenizer(
    [prompt] * len(images),
    padding="max_length",
    max_length=pipe.tokenizer.model_max_length,
    truncation=True,
    return_tensors="pt",
)
with torch.no_grad():
    encoder_hidden_states = pipe.text_encoder(text_inputs.input_ids.to("cuda"))[0]

In [ ]:
from diffusers import StableDiffusionPipeline, DDPMScheduler
import torch
from tqdm import tqdm

# Load pipeline
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16
).to("cuda")

from diffusers.models.attention_processor import AttnProcessor2_0, LoRAAttnProcessor2_0
from peft import LoraConfig

# from diffusers.utils import is_peft_available

# Create a proper PEFT LoRA config
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["to_q", "to_k", "to_v", "to_out.0"],
    lora_dropout=0.1,
    bias="none",
    task_type="UNET"
)

# Now correctly add the adapter
pipe.unet.add_adapter(lora_config, adapter_name="ukiyoe-lora")
pipe.unet.set_adapter("ukiyoe-lora")
pipe.unet.train()

# Optimizer
optimizer = torch.optim.AdamW(pipe.unet.parameters(), lr=1e-4)

# Noise scheduler
noise_scheduler = DDPMScheduler.from_pretrained(model_id, subfolder="scheduler")

# ✅ Checkpoint saving setup
checkpoint_base_dir = "/content/drive/MyDrive/ukiyoe_lora/checkpoints"
os.makedirs(checkpoint_base_dir, exist_ok=True)

# ✅ Training loop
num_epochs = 20
for epoch in range(num_epochs):
    for i in tqdm(range(len(images))):
        image = images[i].unsqueeze(0)
        latents = pipe.vae.encode(image).latent_dist.sample() * 0.18215

        noise = torch.randn_like(latents)
        timesteps = torch.randint(
            0, noise_scheduler.config.num_train_timesteps, (1,), device=latents.device
        ).long()
        noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

        # Forward pass through UNet
        model_pred = pipe.unet(
            sample=noisy_latents,
            timestep=timesteps,
            encoder_hidden_states=encoder_hidden_states[i].unsqueeze(0)
        ).sample

        loss = torch.nn.functional.mse_loss(model_pred, noise)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        torch.cuda.empty_cache()


        # ✅ Save LoRA checkpoint every 5 epochs
    if (epoch + 1) % 5 == 0:
        checkpoint_path = os.path.join(checkpoint_base_dir, f"epoch_{epoch+1}")
        os.makedirs(checkpoint_path, exist_ok=True)
        pipe.unet.save_pretrained(checkpoint_path)
        print(f"✅ Checkpoint saved at: {checkpoint_path}")

print("🎉 Training complete.")

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

✅ Checkpoint saved at: /content/drive/MyDrive/ukiyoe_lora/checkpoints/epoch_10


100%|██████████| 1082/1082 [04:04<00:00,  4.42it/s]


✅ Checkpoint saved at: /content/drive/MyDrive/ukiyoe_lora/checkpoints/epoch_15


100%|██████████| 1082/1082 [04:02<00:00,  4.46it/s]


✅ Checkpoint saved at: /content/drive/MyDrive/ukiyoe_lora/checkpoints/epoch_20
🎉 Training complete.


In [ ]:
# ✅ 8. Save LoRA weights
output_dir = "/content/drive/MyDrive/ukiyoe_lora/lora_weights"
os.makedirs(output_dir, exist_ok=True)
unet.save_pretrained(output_dir)

print(f"🎉 LoRA weights saved to: {output_dir}")

🎉 LoRA weights saved to: /content/drive/MyDrive/ukiyoe_lora/lora_weights
